# Summarization with fine-tuned summarization models

In [1]:
import gc
import torch
from tqdm.notebook import tqdm
from utils import SummarizationPipeline, ModelConfig, LoggingConfig, load_all_available_transcripts, TextChunker

/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:
transcripts = load_all_available_transcripts()
transcripts = transcripts[:3]

2025-04-30 11:45:32,879 - utils.loaders - INFO - 32307


2025-04-30 11:45:33,599 - utils.loaders - INFO - Successfully loaded local transcripts for 32307
2025-04-30 11:45:33,600 - utils.loaders - INFO - Successfully loaded 32307.csv
2025-04-30 11:45:33,601 - utils.loaders - INFO - 126475
2025-04-30 11:45:33,711 - utils.loaders - INFO - Successfully loaded local transcripts for 126475
2025-04-30 11:45:33,712 - utils.loaders - INFO - Successfully loaded 126475.csv
2025-04-30 11:45:33,712 - utils.loaders - INFO - 26446
2025-04-30 11:45:33,759 - utils.loaders - INFO - Successfully loaded local transcripts for 26446
2025-04-30 11:45:33,760 - utils.loaders - INFO - Successfully loaded 26446.csv
2025-04-30 11:45:33,761 - utils.loaders - INFO - 388904
2025-04-30 11:45:33,847 - utils.loaders - INFO - Successfully loaded local transcripts for 388904
2025-04-30 11:45:33,847 - utils.loaders - INFO - Successfully loaded 388904.csv
2025-04-30 11:45:33,848 - utils.loaders - INFO - 312932093
2025-04-30 11:45:33,885 - utils.loaders - INFO - Successfully load

In [3]:
transcripts = transcripts.full_text.tolist()

In [4]:
checkpoints = ['facebook/bart-large-cnn', 'google-t5/t5-base', 'google/pegasus-x-large', 'human-centered-summarization/financial-summarization-pegasus']

local_paths = [f'../models/ragsum-{checkpoint}-billsum' for checkpoint in checkpoints]

logging_config: LoggingConfig = LoggingConfig()

summaries = []

for checkpoint, path in zip(checkpoints, local_paths):

    model_config: ModelConfig = ModelConfig(
        model_name_or_path=checkpoint, device='cuda' if torch.cuda.is_available() else 'cpu'
    )

    pipeline = SummarizationPipeline(model_config=model_config, logging_config=logging_config, remote=False)
    pipeline.load_from_local(path)
    
    tokenizer = pipeline.get_tokenizer()
    chunker = TextChunker(tokenizer)

    try:
        for transcript in tqdm(transcripts):
            chunks = chunker.chunk_text(transcript)
            chunk_summaries = [pipeline.summarize(chunk) for chunk in chunks]
            combined_summary = " ".join(chunk_summaries)
            max_rounds = 5
            round_count = 0
            final_summary = ""
            while round_count < max_rounds:
                print(f'round {round_count+1}')
                input_ids = tokenizer(combined_summary, return_tensors='pt', truncation=False)['input_ids']
                if input_ids.shape[1] <= min(1024, pipeline.model_max_length):
                    final_summary = combined_summary
                    break
                re_chunks = chunker.chunk_text(combined_summary)
                re_chunk_summaries = [pipeline.summarize(chunk) for chunk in re_chunks]
                combined_summary = " ".join(re_chunk_summaries)
                round_count += 1
            else:
                final_summary = combined_summary
            summaries.append(final_summary)
    finally:
        del pipeline
        del model_config
        torch.cuda.empty_cache()
        gc.collect()


2025-04-30 11:45:38,113 - SummarizationPipeline - INFO - Initializing pipeline with model facebook/bart-large-cnn
2025-04-30 11:45:38,115 - SummarizationPipeline - INFO - Loading model and tokenizer from local path: ../models/ragsum-facebook/bart-large-cnn-billsum
/workspace/persistent/ragsum/venv/lib/python3.10/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
2025-04-30 11:46:03,279 - SummarizationPipeline - INFO - Local load complete. Model is standard type, 1024 max length.
2025-04-30 11:46:03,654 - utils.text_chunker - INFO - Initialized TextChunker with chunk_size=1024, chunk_overlap=102, prefix=""
2025-04-30 11:46:03,655 - utils.text_chunker - INFO - Starting text chunking...
Chunking text: 100%|██████████| 13/13 [00:00<00:00, 197557.80it/s]
2025-04-30 11:46:03,662 - utils.text_chunker - INFO

round 1


Chunking text: 100%|██████████| 2/2 [00:00<00:00, 35394.97it/s]
2025-04-30 11:46:20,050 - utils.text_chunker - INFO - Text successfully split into 2 chunks.
2025-04-30 11:46:20,051 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:21,005 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:21,912 - utils.text_chunker - INFO - Starting text chunking...


round 2


Chunking text: 100%|██████████| 16/16 [00:00<00:00, 262144.00it/s]
2025-04-30 11:46:21,921 - utils.text_chunker - INFO - Text successfully split into 16 chunks.
2025-04-30 11:46:21,921 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:22,876 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:23,830 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:24,785 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:25,691 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:26,289 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:27,105 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:27,906 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:46:28,879 - SummarizationPipeline - INFO - Genera

...

round 2


Chunking text: 100%|██████████| 47/47 [00:00<00:00, 531353.88it/s]
2025-04-30 11:51:47,871 - utils.text_chunker - INFO - Text successfully split into 47 chunks.
2025-04-30 11:51:47,872 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:48,973 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:49,739 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:50,610 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:51,345 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:52,194 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:52,991 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:53,833 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:51:54,585 - SummarizationPipeline - INFO - Genera

round 1


Chunking text: 100%|██████████| 6/6 [00:00<00:00, 102717.65it/s]
2025-04-30 11:52:26,433 - utils.text_chunker - INFO - Text successfully split into 6 chunks.
2025-04-30 11:52:26,434 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:52:27,461 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:52:28,257 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:52:29,214 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:52:29,988 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)
2025-04-30 11:52:30,737 - SummarizationPipeline - INFO - Generating summary (max_new_tokens=100)


round 2


In [5]:
summaries

["JPMorgan's 21st Annual Technology and Automotive Investor Forum is held at the Consumer Electronics Show (CES) in Las Vegas, Nevada. JPMorgan is a sponsor of the conference, and the conference is open to the public. JPMorgan's semiconductor and semiconductor capital equipment analyst Harlan Sur:  Oren architecture has been adopted by Mercedes in their next-generation factories similar to some of the other automotive companies that are really seeing the need of the digital twin of the Ampere:  have seen is probably refreshes of the most important gamers out there to be a little bit over three years.  I think we'll see both new gamers and that refresh.  Siemens:  We've already probably passed an inflection point that we're seeing the growth of Orin with our NEVs. The next phase will be coming with our design wins both on robotaxis and bringing some of the early AV to market.",
 "JPMorgan's 41st Annual Healthcare Conference:  Presenter:  Kimberly Powell, Vice President of Healthcare at 